# Data Checker
a program that we can run the data through to check stuff - just need to update the "data" line to use the most recent csv

Things to check:
- all latitudes and longitudes are legit values
- that there are no repeats of photoID
- that there are no repeats of URL
- Check that URLs are all legit (200ok response or smth)
- that AlbumID and AlbumTitle match
- lat and long match the county
- lat and long match the state
- county matches state

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import validators
import re

C:\Users\csky2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# importing flickr data
data = pd.read_csv('tidied_threaded_data_pull.csv')

## 1. Latitude and longitude are legit values.
Latitude should be between -90, 90.
Longitude should be between -180, 180.

In [3]:
lat_data = data['Latitude'].dropna()
print('Number of invalid latitudes:', lat_data[lat_data.between(-90, 90) == False].sum())

Number of invalid latitudes: 0.0


In [4]:
lon_data = data['Longitude'].dropna()
print('Number of invalid longitudes:', lon_data[lon_data.between(-180, 80) == False].sum())

Number of invalid longitudes: 0.0


## 2. There are no repeats of PhotoID.

In [5]:
print('Number of repeat PhotoIDs:', data['PhotoID'].value_counts()[data['PhotoID'].value_counts() > 1].sum())

Number of repeat PhotoIDs: 0


## 3. There are no repeats of URL.

In [6]:
print('Number of repeat URLs:', data['URL'].value_counts()[data['URL'].value_counts() > 1].sum())

Number of repeat URLs: 0


## 4. URL calls are valid.
Faster to use the validators library than loop to check 200 responses (https://validators.readthedocs.io/en/latest/).

In [7]:
def is_valid_url(url):
    return validators.url(url)

url_checker_df = pd.DataFrame({'URL': data['URL'], 'check': data['URL'].apply(is_valid_url)})

print('Number of invalid URLs:', url_checker_df[url_checker_df['check'] == False]['check'].sum())

Number of invalid URLs: 0


## 5. AlbumIDs match AlbumTitles.

In [8]:
# need to update the scrape to include albumID

## 6. The latitude and longitude point match the county.

In [9]:
# reading county and state shapefiles to check location matches
# https://www.census.gov/cgi-bin/geo/shapefiles/index.php
counties_gdf = gpd.read_file('tl_2023_us_county/tl_2023_us_county.shp') 
states_gdf = gpd.read_file('tl_2023_us_state/tl_2023_us_state.shp')

In [10]:
# use to test long/lat county/state checker - ONLY FOR TESTING CODE, DELETE WHEN RUNNING CHECKER ON FINAL DATASET
data = pd.read_csv('attribute_table.csv')
data.drop_duplicates(subset=['Latitude', 'Longitude', 'County', 'State'], inplace=True)

In [11]:
# defining helper functions
def process_area(area):
    if area not in ["County", "State"]:
         raise ValueError("Parameter 'area' must be 'County' or 'State'")
        
    # Function logic here
    if area == "County":
        gdf_name = 'NAME'
        print("Processing County Matches")
    elif area == "State":
        gdf_name = 'STUSPS'
        print("Processing State Matches")
    return gdf_name

def fix_text(phrase):
    name = re.sub(r'county', '', phrase, flags=re.IGNORECASE).strip()
    name = re.sub(r'[^a-zA-Z]', '', name)
    return name.strip()

In [12]:
def location_match(flickr_data, gpd_df, area):

    gdf_name = process_area(area)

    locations = flickr_data[['Latitude', 'Longitude', area]]

    geometry = [Point(xy) for xy in zip(locations['Longitude'], locations['Latitude'])]
    geo_data = gpd.GeoDataFrame(locations, geometry=geometry)
    geo_data = geo_data.set_crs(gpd_df.crs)

    county_latlon_join = gpd.sjoin(geo_data, gpd_df, how='left', op='within')[[area, gdf_name]]

    county_latlon_join[area] = county_latlon_join[area].astype('str').apply(fix_text)
    county_latlon_join[gdf_name] = county_latlon_join[gdf_name].astype('str').apply(fix_text)

    county_latlon_join['match'] = county_latlon_join[area].str.lower() == county_latlon_join[gdf_name].str.lower()

    nan_mismatches = county_latlon_join[(county_latlon_join['match'] == False) & ((county_latlon_join[area] == 'nan') | (county_latlon_join[gdf_name] == 'nan'))]

    name_mismatches = county_latlon_join[county_latlon_join['match'] == False].drop(index = nan_mismatches.index)

    return nan_mismatches, name_mismatches

In [13]:
county_nan_mismatches, county_name_mismatches = location_match(data, counties_gdf, 'County')

Processing County Matches


C:\Users\csky2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py:3398: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
print('Number of county-coordinate mismatches due to missing data:', county_nan_mismatches.shape[0])
print('Number of county-coordinate name mismatches:', county_name_mismatches.shape[0])
print('Percent of data where counties are mismatched:', (county_nan_mismatches.shape[0] + county_name_mismatches.shape[0])/data.shape[0]*100)

Number of county-coordinate mismatches due to missing data: 1299
Number of county-coordinate name mismatches: 93
Percent of data where counties are mismatched: 68.70681145113524


## 7. The latitude and longitude point match the state.

In [15]:
state_nan_mismatches, state_name_mismatches =location_match(data, states_gdf, 'State')

Processing State Matches


C:\Users\csky2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py:3398: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
print('Number of state-coordinate mismatches due to missing data:', state_nan_mismatches.shape[0])
print('Number of state-coordinate name mismatches:', state_name_mismatches.shape[0])
print('Percent of data where states are mismatched:', (state_nan_mismatches.shape[0] + state_name_mismatches.shape[0])/data.shape[0]*100)

Number of state-coordinate mismatches due to missing data: 45
Number of state-coordinate name mismatches: 83
Percent of data where states are mismatched: 6.317867719644619


## 8. The county and state match each other.

In [17]:
counties_gdf['STATEFP'] = counties_gdf['STATEFP'].astype('int')
states_gdf['STATEFP'] = states_gdf['STATEFP'].astype('int')

def fix_text2(phrase):
    if type(phrase) != int:
        name = re.sub(r'county', '', str(phrase), flags=re.IGNORECASE).strip()
        name = re.sub(r'[^a-zA-Z]', '', name)
        rtn = name.strip()
    else:
        rtn = phrase
    return rtn

In [18]:
counties = counties_gdf[['NAME', 'STATEFP']].apply(lambda x: x.apply(fix_text2)).rename(columns= {'NAME': 'County', 'STATEFP': 'StateID'})
states = states_gdf[['STUSPS', 'STATEFP']].apply(lambda x: x.apply(fix_text2)).rename(columns= {'STUSPS': 'State', 'STATEFP': 'StateID'})
area_data = data[['County', 'State']].apply(lambda x: x.apply(fix_text2))

In [19]:
merged_df = pd.merge(counties, states, on='StateID', how='left')

In [20]:
# multi_index_area = pd.MultiIndex.from_frame(area_data[['County', 'State']])
multi_index_merged = pd.MultiIndex.from_frame(merged_df[['County', 'State']])
area_data['match'] = area_data.set_index(['County', 'State']).index.isin(multi_index_merged)

In [21]:
mismatch_county_state = area_data[(area_data['match'] == False) & (area_data['County'] != 'nan') & (area_data['State'] != 'nan')]
print('Number of photos with mismatched county and state:', mismatch_county_state.shape[0])
print('Percent of photos with mismatched county and state:', mismatch_county_state.shape[0]/data.shape[0]*100)

Number of photos with mismatched county and state: 58
Percent of photos with mismatched county and state: 2.8627838104639687
